In [37]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision      # 数据库模块
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
import numpy as np
import os
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [43]:
torch.manual_seed(1)    # reproducible

# Hyper Parameters
EPOCH = 100           # 训练整批数据多少次, 为了节约时间, 我们只训练一次
BATCH_SIZE = 50
LR = 0.001          # 学习率
DOWNLOAD_MNIST = False  # 如果你已经下载好了mnist数据就写上 False

train_data = torchvision.datasets.MNIST(
    root='./mnist/',    # 保存或者提取位置
    train=True,  # this is training data
    transform=torchvision.transforms.ToTensor(),    # 转换 PIL.Image or numpy.ndarray 成
                                                    # torch.FloatTensor (C x H x W), 训练的时候 normalize 成 [0.0, 1.0] 区间
    download=DOWNLOAD_MNIST,          # 没下载就下载, 下载了就不用再下了
)
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)

# 批训练 50samples, 1 channel, 28x28 (50, 1, 28, 28)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)


# 为了节约时间, 我们测试时只测试前2000个
test_x = torch.unsqueeze(test_data.data, dim=1).type(torch.FloatTensor)[:2000]/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.targets[:2000]

In [17]:
class vgg_layer(nn.Module):
    def __init__(self, nin, nout):
        super(vgg_layer, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nin, nout, 3, 1, 1),
            nn.BatchNorm2d(nout),
            nn.LeakyReLU(0.2)
        )

    def forward(self, input):
        return self.main(input)
class dcgan_conv(nn.Module):
    def __init__(self, nin, nout):
        super(dcgan_conv, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nin, nout, 4, 2, 1
                      ),
            nn.BatchNorm2d(nout),
            nn.LeakyReLU(0.2),
        )

    def forward(self, input):
        return self.main(input)
class dcgan_upconv(nn.Module):
    def __init__(self, nin, nout):
        super(dcgan_upconv, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nin, nout, 4, 2, 1, ),
            nn.BatchNorm2d(nout),
            nn.LeakyReLU(0.2),
        )

    def forward(self, input):
        return self.main(input)

In [101]:
class encoder(nn.Module):
    def __init__(self):
        super(encoder,self).__init__()
        self.main = nn.Sequential(
            dcgan_conv(1,2),
            dcgan_conv(2,4),
            dcgan_conv(4,8),
            #dcgan_conv(8,16)
        )
        self.flatten=nn.Sequential(
            nn.Linear(72,10),
            nn.ReLU(),
            nn.BatchNorm1d(10)
        )
    def forward(self,input):
        out = self.main(input)
        out = out.view(out.size(0),-1)
        out = self.flatten(out)
        return out

In [102]:
test = encoder()
test=test.cuda()

In [106]:
optimizer=torch.optim.Adam(test.parameters(),lr=0.001)
scheduler=StepLR(optimizer,step_size=5,gamma=0.8)
criterion=nn.MSELoss()
epochs=5  
for epoch 
for j,(trainData,i) in enumerate(train_loader):
    trainData=trainData.cuda()
    print(i)

tensor([5, 3, 8, 5, 0, 5, 3, 4, 4, 0, 6, 6, 8, 3, 9, 1, 1, 1, 7, 4, 4, 8, 3, 1,
        2, 1, 4, 9, 3, 6, 5, 4, 6, 7, 2, 3, 0, 7, 3, 5, 5, 2, 9, 9, 0, 8, 2, 0,
        9, 0])
tensor([0, 9, 1, 3, 9, 9, 3, 6, 5, 0, 4, 6, 4, 4, 8, 8, 4, 8, 9, 2, 3, 5, 8, 0,
        0, 5, 9, 3, 5, 3, 5, 4, 4, 8, 2, 1, 5, 2, 1, 0, 2, 4, 2, 8, 7, 4, 0, 6,
        0, 5])
tensor([0, 2, 7, 6, 0, 1, 5, 1, 5, 1, 3, 0, 8, 2, 3, 9, 6, 6, 0, 8, 5, 2, 8, 4,
        5, 7, 3, 8, 6, 7, 8, 5, 0, 0, 7, 2, 3, 8, 7, 7, 0, 9, 3, 8, 5, 6, 3, 7,
        2, 3])
tensor([7, 8, 5, 1, 5, 2, 0, 7, 4, 8, 6, 0, 4, 6, 8, 8, 2, 3, 7, 4, 9, 4, 3, 4,
        1, 6, 6, 3, 9, 0, 4, 0, 6, 8, 1, 4, 8, 0, 2, 1, 0, 4, 0, 4, 7, 5, 8, 7,
        6, 6])
tensor([7, 2, 6, 8, 4, 6, 7, 1, 1, 6, 5, 7, 6, 2, 6, 7, 0, 6, 0, 4, 1, 6, 6, 3,
        8, 2, 0, 2, 9, 1, 4, 5, 8, 2, 3, 3, 3, 3, 5, 3, 2, 4, 1, 2, 5, 2, 8, 7,
        0, 9])
tensor([1, 9, 1, 6, 8, 7, 5, 8, 9, 4, 9, 5, 9, 9, 0, 9, 2, 0, 7, 1, 1, 3, 9, 5,
        5, 4, 1, 6, 2, 5, 4, 5, 8, 9, 6, 1, 6

tensor([4, 7, 2, 3, 0, 4, 9, 9, 3, 7, 4, 9, 7, 5, 2, 9, 0, 1, 7, 7, 1, 8, 4, 9,
        7, 9, 8, 6, 9, 6, 5, 4, 4, 8, 1, 0, 1, 0, 1, 0, 4, 4, 4, 5, 2, 0, 6, 8,
        8, 8])
tensor([3, 8, 2, 6, 6, 2, 8, 2, 9, 0, 5, 9, 3, 0, 2, 2, 7, 7, 5, 4, 0, 3, 2, 2,
        8, 1, 4, 7, 5, 5, 5, 1, 1, 7, 1, 1, 4, 0, 4, 4, 8, 4, 2, 0, 9, 1, 3, 3,
        6, 4])
tensor([7, 7, 7, 8, 3, 6, 2, 6, 1, 9, 9, 3, 7, 9, 9, 9, 9, 6, 0, 4, 9, 0, 0, 3,
        3, 9, 5, 4, 8, 0, 1, 9, 6, 0, 2, 9, 8, 2, 9, 6, 9, 3, 7, 1, 0, 1, 6, 7,
        3, 1])
tensor([1, 6, 3, 6, 9, 3, 8, 6, 5, 4, 5, 0, 5, 7, 0, 7, 1, 6, 2, 4, 8, 3, 1, 2,
        6, 6, 6, 7, 3, 1, 8, 4, 8, 1, 4, 3, 8, 1, 1, 8, 4, 7, 6, 0, 9, 6, 5, 9,
        2, 3])
tensor([8, 6, 8, 2, 3, 7, 6, 4, 4, 0, 1, 6, 0, 7, 9, 7, 6, 5, 3, 7, 2, 1, 4, 5,
        8, 6, 7, 2, 2, 5, 7, 2, 0, 6, 7, 8, 2, 5, 6, 1, 6, 1, 7, 8, 4, 3, 1, 3,
        5, 5])
tensor([2, 2, 9, 1, 2, 6, 2, 9, 4, 3, 6, 6, 0, 4, 6, 4, 9, 5, 2, 8, 9, 7, 3, 8,
        4, 5, 8, 3, 2, 3, 0, 2, 5, 9, 9, 7, 2

tensor([9, 0, 0, 8, 8, 7, 0, 4, 4, 4, 9, 5, 3, 7, 8, 7, 9, 7, 6, 0, 4, 8, 1, 9,
        7, 1, 3, 8, 8, 6, 5, 8, 3, 5, 7, 4, 7, 2, 0, 4, 8, 8, 2, 4, 0, 4, 1, 4,
        5, 0])
tensor([2, 8, 9, 0, 2, 6, 3, 6, 6, 9, 7, 5, 0, 3, 8, 4, 2, 9, 7, 9, 2, 0, 1, 9,
        5, 1, 8, 2, 2, 5, 9, 2, 1, 3, 1, 8, 5, 3, 5, 3, 4, 5, 0, 8, 0, 5, 2, 5,
        0, 9])
tensor([0, 8, 7, 2, 3, 3, 4, 2, 5, 4, 6, 0, 5, 7, 8, 6, 8, 4, 5, 9, 6, 8, 8, 8,
        4, 8, 5, 7, 6, 8, 9, 5, 9, 4, 1, 4, 0, 2, 7, 5, 4, 8, 2, 6, 5, 6, 4, 6,
        1, 2])
tensor([1, 1, 8, 0, 1, 3, 9, 5, 1, 0, 9, 8, 1, 0, 5, 6, 0, 8, 6, 1, 0, 9, 7, 6,
        2, 3, 4, 6, 6, 1, 7, 9, 1, 1, 1, 9, 7, 0, 6, 7, 4, 6, 5, 2, 3, 7, 7, 6,
        6, 2])
tensor([8, 6, 8, 2, 3, 6, 1, 9, 8, 3, 3, 1, 1, 9, 2, 8, 1, 5, 6, 5, 4, 0, 7, 4,
        5, 9, 0, 4, 7, 4, 6, 6, 5, 8, 4, 6, 0, 2, 5, 5, 1, 9, 8, 0, 0, 5, 4, 9,
        4, 7])
tensor([9, 7, 7, 1, 3, 4, 6, 4, 4, 1, 1, 1, 7, 4, 8, 3, 7, 6, 6, 9, 2, 3, 2, 9,
        6, 8, 1, 9, 9, 2, 8, 4, 1, 7, 2, 4, 0

tensor([7, 5, 3, 0, 3, 9, 1, 7, 1, 8, 8, 4, 0, 5, 1, 0, 1, 7, 7, 1, 8, 1, 6, 0,
        4, 3, 9, 9, 6, 5, 9, 4, 9, 6, 8, 3, 6, 0, 9, 4, 1, 9, 1, 1, 0, 6, 4, 9,
        1, 4])
tensor([6, 3, 0, 0, 1, 5, 0, 6, 9, 7, 9, 2, 3, 0, 5, 7, 5, 2, 8, 3, 7, 1, 0, 8,
        6, 8, 4, 6, 2, 5, 4, 7, 0, 4, 4, 7, 8, 4, 0, 5, 3, 8, 4, 4, 7, 4, 4, 2,
        2, 1])
tensor([8, 1, 9, 6, 4, 6, 3, 5, 2, 2, 8, 9, 8, 3, 4, 4, 8, 1, 9, 5, 1, 9, 8, 2,
        1, 1, 4, 7, 6, 3, 1, 7, 5, 5, 4, 4, 6, 2, 0, 2, 7, 4, 3, 2, 1, 7, 9, 6,
        3, 1])
tensor([0, 0, 2, 0, 3, 6, 7, 1, 7, 0, 6, 2, 3, 6, 8, 0, 3, 1, 0, 9, 4, 5, 1, 3,
        1, 3, 8, 3, 9, 4, 5, 7, 4, 9, 3, 1, 9, 4, 9, 7, 0, 0, 9, 0, 1, 1, 9, 8,
        8, 3])
tensor([8, 5, 8, 8, 8, 5, 9, 2, 4, 6, 0, 6, 1, 3, 1, 2, 9, 4, 4, 0, 0, 4, 4, 5,
        0, 3, 9, 7, 6, 6, 3, 4, 2, 8, 5, 2, 1, 2, 2, 4, 7, 4, 9, 1, 7, 4, 7, 5,
        8, 9])
tensor([1, 0, 9, 3, 1, 0, 9, 6, 0, 1, 8, 8, 8, 9, 3, 0, 6, 1, 7, 0, 6, 8, 1, 0,
        9, 6, 7, 4, 6, 9, 6, 1, 4, 2, 1, 0, 8

tensor([6, 1, 6, 1, 8, 1, 8, 2, 9, 5, 5, 9, 7, 9, 8, 5, 4, 1, 9, 1, 0, 4, 3, 3,
        8, 0, 6, 3, 5, 1, 8, 9, 4, 1, 4, 2, 5, 3, 5, 8, 6, 4, 0, 3, 3, 6, 2, 6,
        7, 2])
tensor([0, 1, 7, 1, 9, 1, 9, 9, 6, 3, 4, 1, 2, 7, 3, 8, 2, 2, 2, 2, 1, 9, 9, 1,
        9, 6, 1, 4, 7, 4, 9, 2, 2, 7, 1, 1, 6, 1, 9, 8, 7, 4, 1, 0, 2, 3, 0, 8,
        4, 6])
tensor([3, 5, 2, 6, 2, 5, 0, 2, 2, 7, 5, 4, 8, 4, 3, 1, 4, 7, 8, 2, 9, 5, 9, 2,
        2, 6, 5, 8, 9, 8, 7, 8, 4, 1, 8, 0, 9, 8, 6, 9, 7, 5, 3, 7, 9, 4, 0, 3,
        5, 3])
tensor([4, 1, 5, 8, 3, 2, 6, 1, 8, 2, 3, 1, 2, 3, 2, 3, 8, 4, 6, 3, 5, 3, 9, 2,
        5, 8, 8, 8, 9, 1, 9, 0, 2, 2, 1, 8, 3, 3, 4, 2, 1, 2, 7, 7, 0, 1, 3, 3,
        4, 8])
tensor([4, 6, 2, 0, 5, 7, 6, 7, 7, 9, 3, 5, 2, 5, 5, 6, 0, 2, 9, 0, 1, 7, 4, 7,
        4, 1, 0, 4, 9, 9, 1, 5, 4, 2, 4, 3, 1, 3, 8, 1, 7, 0, 9, 8, 4, 0, 7, 2,
        9, 0])
tensor([4, 6, 7, 9, 8, 6, 3, 5, 6, 6, 0, 2, 6, 1, 0, 7, 1, 5, 7, 8, 6, 3, 9, 0,
        5, 1, 5, 8, 9, 5, 1, 9, 7, 3, 4, 9, 0

...

tensor([4, 0, 1, 1, 0, 6, 5, 5, 8, 4, 3, 7, 1, 3, 3, 9, 0, 1, 8, 7, 6, 9, 9, 8,
        3, 5, 9, 1, 4, 7, 9, 0, 8, 8, 7, 5, 4, 1, 9, 7, 4, 9, 6, 2, 1, 3, 2, 2,
        6, 8])
tensor([7, 3, 1, 6, 6, 9, 6, 4, 6, 0, 0, 4, 2, 3, 0, 9, 0, 4, 6, 7, 2, 9, 3, 0,
        9, 7, 4, 8, 3, 5, 6, 9, 3, 1, 6, 9, 2, 6, 2, 4, 6, 7, 8, 7, 1, 5, 4, 1,
        9, 7])
tensor([2, 9, 2, 2, 0, 7, 6, 9, 8, 2, 0, 3, 6, 0, 9, 2, 6, 9, 6, 8, 7, 1, 9, 5,
        2, 9, 2, 2, 6, 0, 0, 1, 2, 3, 3, 6, 2, 6, 9, 1, 0, 0, 0, 4, 9, 8, 5, 3,
        9, 7])
tensor([6, 9, 8, 8, 0, 7, 5, 9, 2, 4, 4, 7, 8, 4, 5, 2, 7, 7, 0, 4, 2, 1, 9, 4,
        4, 8, 3, 6, 2, 8, 6, 0, 0, 4, 6, 3, 5, 6, 4, 5, 8, 1, 3, 9, 5, 5, 2, 0,
        8, 5])
tensor([3, 1, 1, 5, 6, 0, 0, 7, 4, 5, 9, 4, 1, 9, 7, 0, 0, 6, 9, 6, 0, 7, 8, 1,
        5, 6, 1, 9, 8, 2, 4, 2, 7, 1, 6, 5, 4, 0, 1, 5, 5, 8, 5, 1, 9, 7, 0, 7,
        5, 9])
tensor([6, 9, 8, 0, 1, 8, 1, 1, 0, 7, 9, 6, 4, 8, 8, 1, 5, 6, 0, 7, 6, 1, 2, 0,
        9, 4, 3, 0, 5, 3, 0, 9, 2, 6, 0, 8, 2

tensor([1, 3, 2, 4, 7, 3, 9, 4, 6, 6, 6, 2, 1, 3, 8, 6, 6, 0, 4, 3, 1, 4, 2, 6,
        9, 7, 3, 9, 0, 1, 3, 7, 4, 5, 4, 7, 6, 9, 8, 2, 7, 2, 9, 6, 2, 4, 8, 9,
        8, 8])
tensor([7, 7, 0, 1, 2, 5, 8, 0, 7, 3, 6, 5, 0, 3, 0, 1, 7, 9, 7, 4, 3, 1, 5, 6,
        9, 5, 7, 3, 7, 2, 9, 7, 9, 6, 9, 1, 0, 0, 4, 4, 9, 8, 3, 5, 4, 8, 1, 4,
        2, 1])
tensor([4, 7, 9, 7, 9, 5, 3, 4, 8, 1, 1, 2, 8, 8, 9, 6, 9, 6, 3, 5, 1, 6, 2, 3,
        1, 3, 8, 7, 8, 5, 1, 9, 8, 2, 0, 2, 0, 9, 7, 8, 6, 4, 7, 7, 0, 9, 5, 3,
        0, 6])
tensor([9, 9, 0, 5, 0, 9, 9, 1, 7, 1, 4, 5, 9, 0, 7, 9, 2, 3, 3, 2, 2, 1, 0, 2,
        0, 6, 4, 5, 7, 0, 1, 7, 3, 4, 2, 6, 0, 0, 3, 6, 6, 9, 7, 0, 0, 4, 7, 7,
        1, 7])
tensor([6, 7, 2, 3, 6, 4, 8, 7, 4, 7, 1, 5, 0, 1, 6, 6, 7, 6, 0, 8, 8, 2, 2, 5,
        4, 8, 3, 3, 9, 2, 1, 6, 6, 0, 3, 9, 2, 8, 9, 7, 2, 7, 8, 4, 6, 7, 2, 1,
        8, 3])
tensor([7, 6, 9, 3, 9, 6, 9, 7, 1, 7, 3, 6, 0, 9, 8, 6, 3, 2, 8, 1, 7, 8, 7, 2,
        4, 6, 3, 2, 6, 5, 5, 4, 9, 7, 4, 2, 4

tensor([6, 3, 7, 4, 8, 4, 6, 8, 2, 0, 4, 7, 8, 8, 8, 1, 8, 8, 3, 8, 3, 1, 6, 2,
        1, 5, 2, 5, 2, 2, 1, 2, 3, 2, 9, 0, 8, 2, 8, 9, 6, 6, 1, 3, 1, 4, 7, 1,
        8, 6])
tensor([9, 5, 0, 8, 8, 3, 8, 1, 7, 1, 1, 4, 8, 7, 2, 2, 8, 8, 8, 7, 4, 0, 4, 7,
        3, 4, 2, 3, 9, 1, 0, 2, 1, 3, 1, 0, 8, 2, 7, 4, 3, 3, 1, 9, 1, 8, 6, 9,
        0, 2])
tensor([4, 1, 3, 3, 2, 5, 8, 0, 2, 0, 9, 4, 2, 7, 1, 2, 5, 1, 1, 2, 0, 0, 8, 5,
        9, 6, 3, 6, 0, 5, 4, 5, 0, 8, 2, 6, 7, 8, 2, 8, 9, 5, 2, 0, 6, 6, 4, 9,
        7, 4])
tensor([5, 9, 8, 5, 8, 0, 7, 8, 3, 3, 2, 7, 1, 5, 6, 5, 7, 0, 6, 7, 3, 1, 8, 9,
        7, 6, 6, 7, 1, 8, 9, 2, 3, 7, 6, 0, 2, 6, 0, 4, 5, 8, 4, 8, 7, 4, 5, 2,
        3, 6])
tensor([2, 4, 9, 5, 0, 6, 1, 0, 9, 8, 2, 5, 6, 2, 6, 8, 1, 0, 9, 7, 5, 6, 9, 2,
        1, 0, 1, 4, 8, 1, 8, 3, 4, 1, 5, 1, 1, 4, 2, 5, 3, 1, 7, 8, 9, 6, 5, 8,
        2, 4])
tensor([0, 2, 2, 8, 2, 8, 1, 4, 7, 3, 2, 6, 1, 1, 7, 7, 3, 9, 6, 6, 5, 4, 8, 2,
        6, 0, 6, 2, 1, 5, 5, 3, 1, 5, 3, 6, 4

tensor([0, 6, 3, 5, 0, 4, 0, 9, 5, 7, 4, 8, 8, 1, 1, 4, 7, 0, 5, 7, 7, 2, 3, 4,
        6, 2, 2, 6, 7, 1, 6, 3, 1, 1, 0, 7, 5, 3, 1, 5, 6, 2, 9, 4, 2, 9, 2, 2,
        8, 6])
tensor([4, 4, 0, 6, 6, 6, 1, 9, 1, 1, 6, 7, 4, 0, 9, 0, 2, 2, 8, 5, 1, 8, 2, 8,
        9, 9, 3, 0, 5, 5, 9, 1, 4, 2, 3, 3, 9, 1, 5, 9, 5, 9, 6, 1, 0, 5, 0, 9,
        7, 7])
tensor([9, 5, 3, 9, 4, 4, 1, 5, 3, 2, 5, 0, 3, 7, 0, 7, 8, 6, 7, 1, 3, 5, 1, 2,
        7, 7, 8, 9, 5, 1, 3, 6, 4, 0, 0, 1, 0, 0, 0, 1, 6, 1, 6, 3, 9, 4, 5, 6,
        4, 7])
tensor([6, 8, 3, 0, 4, 4, 3, 1, 2, 0, 3, 9, 2, 3, 7, 1, 5, 4, 5, 9, 7, 6, 3, 9,
        5, 9, 2, 9, 0, 5, 4, 9, 0, 3, 4, 7, 7, 4, 2, 0, 3, 7, 1, 5, 3, 7, 7, 9,
        9, 8])
tensor([4, 1, 3, 5, 4, 0, 6, 5, 0, 8, 5, 6, 1, 3, 0, 1, 1, 6, 7, 6, 2, 1, 7, 0,
        0, 7, 7, 0, 7, 5, 8, 3, 9, 1, 0, 7, 5, 3, 1, 4, 0, 9, 7, 5, 2, 9, 6, 4,
        8, 5])
tensor([3, 5, 1, 9, 0, 0, 4, 4, 2, 1, 5, 9, 6, 3, 0, 3, 4, 0, 5, 4, 6, 0, 6, 3,
        4, 5, 2, 5, 0, 9, 2, 0, 1, 1, 6, 1, 4

tensor([0, 3, 0, 6, 8, 3, 8, 1, 4, 2, 3, 5, 9, 3, 3, 4, 7, 4, 8, 3, 3, 5, 4, 1,
        1, 3, 0, 4, 3, 5, 7, 9, 3, 7, 3, 2, 6, 9, 6, 0, 7, 9, 7, 0, 1, 1, 5, 1,
        5, 2])
tensor([0, 3, 4, 6, 0, 9, 0, 7, 3, 2, 6, 8, 6, 6, 9, 3, 9, 9, 2, 1, 2, 0, 6, 4,
        2, 4, 8, 9, 2, 4, 4, 1, 9, 4, 1, 2, 2, 1, 4, 8, 3, 2, 6, 9, 3, 4, 1, 3,
        4, 8])
tensor([3, 8, 6, 8, 6, 5, 8, 8, 3, 7, 3, 4, 7, 6, 7, 8, 9, 0, 3, 4, 4, 9, 8, 7,
        6, 7, 3, 5, 8, 8, 8, 0, 4, 1, 9, 0, 4, 9, 2, 5, 6, 7, 3, 4, 2, 3, 6, 7,
        0, 9])
tensor([8, 3, 8, 4, 8, 7, 5, 4, 1, 2, 2, 1, 3, 3, 4, 5, 7, 5, 5, 5, 8, 1, 7, 1,
        6, 7, 4, 5, 9, 3, 6, 9, 9, 4, 5, 4, 2, 3, 8, 6, 0, 0, 8, 7, 5, 5, 0, 3,
        1, 7])
tensor([2, 9, 1, 8, 8, 8, 0, 3, 3, 8, 9, 3, 1, 6, 7, 2, 7, 2, 7, 7, 7, 0, 0, 2,
        1, 8, 9, 5, 7, 5, 9, 5, 3, 4, 4, 3, 5, 5, 1, 8, 1, 6, 0, 0, 2, 4, 0, 7,
        0, 8])
tensor([9, 4, 3, 0, 9, 0, 3, 4, 9, 2, 9, 9, 3, 6, 7, 2, 8, 2, 1, 9, 9, 8, 5, 9,
        9, 3, 6, 9, 0, 0, 4, 9, 6, 4, 4, 8, 9